In [2]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [94]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
COMPANY_NAME = 'Royal Apothecary'
COMPANY_IDENTIFIER = 'RA'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 10/23/2021


In [124]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [17]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [125]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [126]:
# Check for how many packages are excluded from analysis,
# seeing less then 2% is really good
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q, params=ANALYSIS_PARAMS)

Only outgoing: 7
Only incoming: 1540
In and out: 8
In and sold at least once 4074
In and sold many times 3946
Total pkgs: 5672
Wrote result to out/Royal Apothecary_inventory_by_month.xls
Excluded 91 / 5672 packages from consideration (1.60%)
  MANY_INCOMING: 40 times
  MISSING_INCOMING: 51 times


In [140]:
import importlib
importlib.reload(util)

# TODO(dlluncor): Just for debugging to make this faster
INVENTORY_DATES = [TODAY_DATE]

date_to_inventory_packages_dataframe = {}
id_to_history = util.get_histories(d)

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'arrived_ate',
            'product_category_name',
            'product_name',
            'quantity',
            'sold_date',
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

In [132]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

In [145]:
# For debugging individual package histories
import importlib
importlib.reload(util)

# Missing transactions?
# 5189649

# Using grams or missing transactions?

PACKAGE_IDS = [
    '3497652'
]

util.analyze_specific_package_histories(d, PACKAGE_IDS, params=ANALYSIS_PARAMS)


Package 3497652 arrived on 04/07/2020 with quantity 350 and price $625.0. {'package_row_id': '946ee344-1995-4182-922c-a8d9797cfc6a', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'C10-0000596-LIC', 'manifest_number': '0000447208', 'created_date': datetime.date(2020, 4, 7), 'received_datetime': Timestamp('2020-04-07 23:11:56+0000', tz='UTC'), 'shipper_facility_license_number': 'C12-0000175-LIC', 'shipper_facility_name': 'CONNECTED TRANSPORTATION PARTNERS INC.', 'recipient_facility_license_number': 'C10-0000596-LIC', 'recipient_facility_name': 'ROYAL APOTHECARY L.L.C.', 'shipment_type_name': 'Wholesale Manifest', 'shipment_transaction_type': 'Wholesale', 'package_id': '3497652', 'package_label': '1A4060300009F02000075117', 'type': 'transfer_incoming', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Flower', 'product_name': 'Old Pal Ready To Roll 0.5oz Indica', 'package_lab_results_status': 'passed', 'ship

In [141]:
import importlib
importlib.reload(util)

util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe
)

Pct of # inventory matching: 93.35%
Accuracy of quantities: 90.39%
Pct of # inventory packages over-estimated: 6.74%
Pct of # quantity over-estimated: 0.15%
Avg quantity delta: 0.53
Avg quantity: 5.53

Num matching packages: 3325
Num actual packages not computed: 237
Num computed packages not in actual: 240

Computed has these extra package IDs; first 20
3066734; computed quantity 7
3068717; computed quantity 1
3507112; computed quantity 1
7610314; computed quantity 2
11763122; computed quantity 3
3067619; computed quantity 3
3068158; computed quantity 7
5189649; computed quantity 20
3068721; computed quantity 1
9736065; computed quantity 1
8749830; computed quantity 1
2756443; computed quantity 1
3080665; computed quantity 2
10190890; computed quantity 12
9266371; computed quantity 1
3804483; computed quantity 2
10190896; computed quantity 16
3833973; computed quantity 1
3455807; computed quantity 1
3497652; computed quantity 28
5945936; computed quantity 3

Computed is missing these 

In [ ]:
#### Unused, previously used code



####

In [ ]:
# Get dataframe TODAYS_DATE

today_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, TODAY_DATE)
today_inventory_packages_dataframe = pandas.DataFrame(
    today_inventory_package_records,
    columns=[
        'package_id',
        'arrived_date',
        'product_category_name',
        'product_name',
        'current_quantity',
        'sold_date',
    ]
)
today_inventory_packages_dataframe.sort_values('package_id')

In [ ]:
# Some extra analysis on inventory


In [ ]:
incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]
incoming_transfer_packages_dataframe

In [ ]:

import math

for inventory_date, inventory_packages_dataframe in date_to_inventory_packages_dataframe.items():
    print(inventory_date)
    print(f'# of packages in inventory: {len(inventory_packages_dataframe.index)}')

    inventory_with_incoming_transfer_packages_dataframe = inventory_packages_dataframe \
        .astype({'package_id': 'int64'}) \
        .merge(
            incoming_transfer_packages_dataframe.astype({'package_id': 'int64'}),
            on='package_id',
            how='inner',
            suffixes=('_l', '_r')
        )
    #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')

    inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

    total_valuation_cost = 0

    for inventory_with_cost_record in inventory_with_cost_records:
        incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
        if math.isnan(incoming_shipped_price):
            incoming_shipped_price = 0
        incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
        current_quantity = inventory_with_cost_record['Current Quantity']
        total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

    print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
    print('')

In [ ]:
company_inventory_packages_dataframe
company_sales_transactions_dataframe
company_outgoing_transfer_packages_dataframe
company_incoming_transfer_packages_dataframe